In [212]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import imgaug.augmenters as aug

In [213]:
def img_read_resize(files):
    img = cv2.imread(files,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, dsize=(40, 30), interpolation=cv2.INTER_LINEAR)
    return img

In [214]:
folder_instr = []
img_bass = []
img_bass_hihat = []
img_bass_ride = []
img_hihat = []
img_snare = []
img_snare_hihat = []
img_snare_tom_floor = []
img_tom_1 = []
img_tom_1_tom_floor = []
img_tom_2 = []
img_tom_2_tom_floor = []
img_tom_floor = []

DIR_instr = 'C:/Users/ADmin/Desktop/drum_instrument_/'

for i_instr in os.listdir(DIR_instr):
    folder_instr = DIR_instr + i_instr
    for i in os.listdir(folder_instr):
        if i_instr == 'Bass':
            img = img_read_resize(folder_instr+'/'+i)
            img_bass.append(img)
        elif i_instr == 'Bass+Hihat':
            img = img_read_resize(folder_instr+'/'+i)
            img_bass_hihat.append(img)
        elif i_instr == 'Bass+Ride':
            img = img_read_resize(folder_instr+'/'+i)
            img_bass_ride.append(img)
        elif i_instr == 'Hihat':
            img = img_read_resize(folder_instr+'/'+i)
            img_hihat.append(img)   
        elif i_instr == 'Snare':
            img = img_read_resize(folder_instr+'/'+i)
            img_snare.append(img) 
        elif i_instr == 'Snare+Hihat':
            img = img_read_resize(folder_instr+'/'+i)
            img_snare_hihat.append(img) 
        elif i_instr == 'Snare+Tom_floor':
            img = img_read_resize(folder_instr+'/'+i)
            img_snare_tom_floor.append(img)    
        elif i_instr == 'Tom_1':
            img = img_read_resize(folder_instr+'/'+i)
            img_tom_1.append(img)   
        elif i_instr == 'Tom_1+Tom_floor':
            img = img_read_resize(folder_instr+'/'+i)
            img_tom_1_tom_floor.append(img)  
        elif i_instr == 'Tom_2':
            img = img_read_resize(folder_instr+'/'+i)
            img_tom_2.append(img)  
        elif i_instr == 'Tom_2+Tom_floor':
            img = img_read_resize(folder_instr+'/'+i)
            img_tom_2_tom_floor.append(img) 
        elif i_instr == 'Tom_floor':
            img = img_read_resize(folder_instr+'/'+i)
            img_tom_floor.append(img)  

In [215]:
def aug_data(crop_x, crop_y, blurfactor, noise, shear): # crop_x = 30, crop_y = 30,blurfactor = 2.0 
                                                        # noise = 0, shear = -10~10
    
    seq_0 = aug.Sequential([
        aug.Crop(px=(0 ,0 , 0, 0)),
        aug.GaussianBlur(sigma=(blurfactor, blurfactor)),
        aug.AdditiveGaussianNoise(scale=noise*255),
        aug.ShearX((shear,shear))
    ])
    
    seq_1 = aug.Sequential([
        aug.Crop(px=(crop_x,crop_x, 0, 0)),
        aug.GaussianBlur(sigma=(blurfactor, blurfactor)),
        aug.AdditiveGaussianNoise(scale=noise*255),
        aug.ShearX((shear,shear))
    ])
    
    seq_2 = aug.Sequential([
        aug.Crop(px=(0, 0, crop_y, crop_y)),
        aug.GaussianBlur(sigma=(blurfactor, blurfactor)),
        aug.AdditiveGaussianNoise(scale=noise*255),
        aug.ShearX((shear,shear))
    ])

    seq_3 = aug.Sequential([
        aug.Crop(px=(crop_x, crop_x, crop_y, crop_y)),
        aug.GaussianBlur(sigma=(blurfactor, blurfactor)),
        aug.AdditiveGaussianNoise(scale=noise*255),
        aug.ShearX((shear,shear))
    ])
    
    seq_4 = aug.Sequential([
        aug.Crop(px=(crop_x * 2, crop_x * 2, 0, 0)),
        aug.GaussianBlur(sigma=(blurfactor, blurfactor)),
        aug.AdditiveGaussianNoise(scale=noise*255),
        aug.ShearX((shear,shear))
    ])

    seq_5 = aug.Sequential([
        aug.Crop(px=(0, 0, crop_y * 2, crop_y * 2)),
        aug.GaussianBlur(sigma=(blurfactor, blurfactor)),
        aug.AdditiveGaussianNoise(scale=noise*255),
        aug.ShearX((shear,shear))
    ])

    seq_6 = aug.Sequential([
        aug.Crop(px=(crop_x * 2, crop_x * 2, crop_y * 2, crop_y * 2)),
        aug.GaussianBlur(sigma=(blurfactor, blurfactor)),
        aug.AdditiveGaussianNoise(scale=noise*255),
        aug.ShearX((shear,shear))
    ])

    return seq_0, seq_1, seq_2, seq_3, seq_4, seq_5, seq_6

In [139]:
"""
x_ = 30
y_ = 30
blur_ = 3.0
noise_ = 0.5
shear_ = -5

seq_0, seq_1, seq_2, seq_3, seq_4, seq_5, seq_6 = aug_data(x_, y_, blur_, noise_, shear_)

for i in range(len(img_bass)):
    img_aug_0 = seq_0(images=img_bass)
    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_0_' + str(i) + \
                 '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) +\
                '_' + str(shear_) +'.jpg',img_aug_0[i])
    img_aug_1 = seq_1(images=img_bass)
    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_1_' + str(i) + \
                '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                '_' + str(shear_) +'.jpg',img_aug_1[i])
    img_aug_2 = seq_2(images=img_bass)
    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_2_' + str(i) + \
                '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                '_' + str(shear_) +'.jpg',img_aug_2[i])
    img_aug_3 = seq_3(images=img_bass)
    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_3_' + str(i) + \
                '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                '_' + str(shear_) +'.jpg',img_aug_3[i])
    img_aug_4 = seq_4(images=img_bass)
    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_4_' + str(i) + \
                '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                '_' + str(shear_) +'.jpg',img_aug_4[i])
    img_aug_5 = seq_5(images=img_bass)
    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_5_' + str(i) + \
                '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                '_' + str(shear_) +'.jpg',img_aug_5[i])
    img_aug_6 = seq_6(images=img_bass)
    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_6_' + str(i) + \
                '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                '_' + str(shear_) +'.jpg',img_aug_6[i])
"""

"\nx_ = 30\ny_ = 30\nblur_ = 3.0\nnoise_ = 0.5\nshear_ = -5\n\nseq_0, seq_1, seq_2, seq_3, seq_4, seq_5, seq_6 = aug_data(x_, y_, blur_, noise_, shear_)\n\nfor i in range(len(img_bass)):\n    img_aug_0 = seq_0(images=img_bass)\n    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_0_' + str(i) +                  '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) +                '_' + str(shear_) +'.jpg',img_aug_0[i])\n    img_aug_1 = seq_1(images=img_bass)\n    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_1_' + str(i) +                 '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) +                 '_' + str(shear_) +'.jpg',img_aug_1[i])\n    img_aug_2 = seq_2(images=img_bass)\n    cv2.imwrite('C:/Users/ADmin/Desktop/drum_instrument/Bass/'+ 'img_aug_2_' + str(i) +                 '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) +                 '_' + str(shear_) +'.jpg',img_aug_2[i

In [216]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [217]:
nfolder = ['Bass', 'Bass+Hihat', 'Bass+Ride', 'Hihat', 'Snare', 'Snare+Hihat', \
           'Snare+Tom_floor', 'Tom_1', 'Tom_1+Tom_floor', 'Tom_2', 'Tom_2+Tom_floor', 'Tom_floor']

In [218]:
orig_img = [img_bass, img_bass_hihat, img_bass_ride, img_hihat, img_snare,\
            img_snare_hihat, img_snare_tom_floor, img_tom_1, img_tom_1_tom_floor,\
            img_tom_2, img_tom_2_tom_floor, img_tom_floor]

In [219]:
#plt.imshow(img_bass[3])

In [220]:
for n, n_folder in  enumerate(nfolder):
    write_DIR = 'C:/Users/ADmin/Desktop/drum_small_train/'
    createFolder(write_DIR + n_folder)
    
    original = orig_img[n]

    crop_para = [2, 1]#[30, 20, 10]
    blur_para = [0.1, 0]#[3.0, 2.0, 1.0, 0] 
    noise_para = [0.001, 0]#[0.5, 0.3, 0.1, 0]
    shear_para = [-2, 0, 2]#[-10, -7, -4, -1, 0, 1, 4, 7, 10]

    for crop in crop_para:
        x_ = crop
        y_ = crop
        for blur_ in blur_para:
            for noise_ in noise_para:
                for shear_ in shear_para:
                    seq_0, seq_1, seq_2, seq_3, seq_4, seq_5, seq_6 = aug_data(x_, y_, blur_, noise_, shear_)

                    for i in range(len(original)):
                        img_aug_0 = seq_0(images=original)
                        cv2.imwrite(write_DIR + n_folder + '/' + 'img_aug_0_' + str(i) + \
                                     '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) +\
                                    '_' + str(shear_) +'.jpg',img_aug_0[i])
                        img_aug_1 = seq_1(images=original)
                        cv2.imwrite(write_DIR + n_folder  + '/' + 'img_aug_1_' + str(i) + \
                                    '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                                    '_' + str(shear_) +'.jpg',img_aug_1[i])
                        img_aug_2 = seq_2(images=original)
                        cv2.imwrite(write_DIR + n_folder  + '/' + 'img_aug_2_' + str(i) + \
                                    '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                                    '_' + str(shear_) +'.jpg',img_aug_2[i])
                        img_aug_3 = seq_3(images=original)
                        cv2.imwrite(write_DIR + n_folder  + '/' + 'img_aug_3_' + str(i) + \
                                    '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                                    '_' + str(shear_) +'.jpg',img_aug_3[i])
                        img_aug_4 = seq_4(images=original)
                        cv2.imwrite(write_DIR + n_folder  + '/' + 'img_aug_4_' + str(i) + \
                                    '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                                    '_' + str(shear_) +'.jpg',img_aug_4[i])
                        img_aug_5 = seq_5(images=original)
                        cv2.imwrite(write_DIR + n_folder  + '/' + 'img_aug_5_' + str(i) + \
                                    '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                                    '_' + str(shear_) +'.jpg',img_aug_5[i])
                        img_aug_6 = seq_6(images=original)
                        cv2.imwrite(write_DIR + n_folder  + '/' + 'img_aug_6_' + str(i) + \
                                    '_' + str(x_) + '_' + str(y_) + '_' + str(blur_) + '_' + str(noise_) + \
                                    '_' + str(shear_) +'.jpg',img_aug_6[i])

In [221]:
a = len(os.listdir(write_DIR+'Bass'))
b = len(os.listdir(write_DIR+'Bass+Hihat'))
c = len(os.listdir(write_DIR+'Bass+Ride'))
d = len(os.listdir(write_DIR+'Hihat'))
e = len(os.listdir(write_DIR+'Snare'))
f = len(os.listdir(write_DIR+'Snare+Hihat'))
g = len(os.listdir(write_DIR+'Snare+Tom_floor'))
h = len(os.listdir(write_DIR+'Tom_1'))
i = len(os.listdir(write_DIR+'Tom_1+Tom_floor'))
j = len(os.listdir(write_DIR+'Tom_2'))
k = len(os.listdir(write_DIR+'Tom_2+Tom_floor'))
l = len(os.listdir(write_DIR+'Tom_floor'))

print(a,'/', b,'/', c,'/', d,'/', e,'/', f,'/', g,'/', h,'/', i,'/', j,'/', k,'/', l)

672 / 672 / 672 / 672 / 672 / 672 / 672 / 672 / 672 / 672 / 672 / 672


In [222]:
a+b+c+d+e+f+g+h+i+j+k+l

8064